In [2]:
import time
import gurobipy as gp
from gurobipy import *
import xlrd
import numpy as np
import tsp
import pandas as pd
from statistics import mean, stdev
from operator import truediv
from python_tsp.exact import solve_tsp_dynamic_programming
import json

# data import
M = []
sheet = pd.read_csv(r'C:\Users\hp\Desktop\SCM\table_TW2.csv',header=None)
N = len(sheet)
b = 2  # expected shopping time at a stock point

for i in range(0, N):
    row = []
    for j in range(0, N):
        row.append(sheet.iloc[i][j])
    M.append(row)
C = M.copy()
# passage = [1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 16, 18, 20, 23, 25, 27, 30, 31, 32, 33, 34, 37, 39, 41, 44,
#            46, 48, 51, 53, 55, 58, 59, 60, 61, 62]
# master_list = passage.copy()
# master_list.remove(1)

In [3]:
def shortest(c,n, v):
    r = range(len(c))
    permutation, distance = solve_tsp_dynamic_programming(c);
    obj = distance
    t = n*b + 5*obj/v
    return t

In [4]:
Out = pd.DataFrame(columns =['arrival rate','probabilty','itr', 'num_customers', 'mean(norm_speed1)', 'stdev(norm_speed1)', 'mean(norm_speed2)',
           'stdev(norm_speed2)', 'mean(norm_t_added1)', 'stdev(norm_t_added1)', 'mean(norm_t_added2)',
           'stdev(norm_t_added2)', 'contacts', 'mean(run_time)', 'stdev(run_time)'])

In [ ]:


def compute_path(n, d, dx, y, t0):
    n = n-1
    M = 10000
    nodes = range(0, n)
    nodes2 = range(1, n+1)
    nodes3 = range(1,n)
    nodes4 = range(0,n+1)

    maxlen = 0
    for k in range(0, n+1):
        if len(dx[k]) > maxlen:
            maxlen = len(dx[k])

    for k in range(0, n+1):
        while len(dx[k]) < maxlen:
            dx[k].append(0)

    e = range(0, maxlen)
    start_time = time.time()

    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model(env=env) as m1:

            # variables
            t = m1.addVars(n+1, lb=0, name="t") # time of arrival at each node
            u = m1.addVars(n+1, lb=1, ub=n+1, vtype=GRB.INTEGER, name="u") # order of visiting of nodes
            x = m1.addVars(n+1, n+1, vtype=GRB.BINARY, name="x") # if i,j are connected x[i][j]=1 else 0
            p = m1.addVars(n+1, maxlen, lb=0, name="p")
            z = m1.addVars(n+1, maxlen, lb=-gp.GRB.INFINITY, ub=gp.GRB.INFINITY, name="z")
            delta = m1.addVars(n+1, maxlen, vtype=GRB.BINARY, name="delta")
            #obj = sum(c[i][j]*x[i, j] for i in nodes for j in nodes) + 10*sum(delta[i, k] for i in nodes2 for k in e)
            obj = t[n] - t[0] + sum(delta[i, k] for i in nodes2 for k in e)# objective function for time 
            # constraints
            m1.addConstr(u[0] == 1)
            m1.addConstr(t[0] == t0)
            m1.addConstr(u[n] == n+1) # we have to return back to the initial node
            for i in nodes4:
                m1.addConstr(x[i, i] == 0)
            print(n)
            m1.addConstr(x[0,n] == 0) # ve are not allowed to go fom first node to itself (also covered in below constraints)
            for i in nodes:
                m1.addConstr(sum(x[i, j] for j in nodes2) == 1) # only one outgoing edge for 0 to n-1

            for j in nodes2:
                m1.addConstr(sum(x[i, j] for i in nodes) == 1) # only one incoming edge from 1 to n
            for i in nodes2:
                m1.addConstr(x[i,0] == 0)
            for i in nodes:
                m1.addConstr(x[n,i] == 0)
            #m1.addConstr(u[n] - u[0] == n)
            for i in nodes:
                for j in nodes2:
                    m1.addConstr(u[i] - u[j] + (n+1) * x[i, j] <= n)

            for i in nodes:
                for j in nodes2:
                    # m1.addConstr(t[j] - t[i] <= big_M * (1 - x[i, j]) + b + 5.0 * c[i][j] / v)
                    m1.addConstr(t[j]-t[i] <= (d[i][j]+b)*x[i,j] + M*(1-x[i,j]))  # time constraint for travel from one node to another
                    m1.addConstr(t[j]-t[i] >= (d[i][j]+b)*x[i,j] - M*(1-x[i,j]))
                    # m1.addConstr(t[j] - t[i] >= -big_M * (1 - x[i, j]) + b + 5.0 * c[i][j] / v)

            for i in nodes2: #time constraint for vertex 0
                m1.addConstr(t[i] - t[0] >= -M * (1 - x[0, i]) + (d[0][i]+b)*x[0,i])
                m1.addConstr(t[i] - t[0] <= M * (1 - x[0, i]) + (d[0][i]+b)*x[0,i])
            for i in nodes: # time constraint fr vertex n
                m1.addConstr(t[n] - t[i] >= -M * (1 - x[i, n]) + (d[n][i]+b)*x[i,n])
                m1.addConstr(t[n] - t[i] <=  M * (1 - x[i, n]) + (d[n][i]+b)*x[i,n])
                

            for i in nodes3: # only for th interior nodes and not 0,n
                for k in e:
                    if dx[i][k] == 0:
                        m1.addConstr(delta[i, k] == 0)
                    else:
                        m1.addConstr(z[i, k] == (t[i] - dx[i][k]) / b)
                        m1.addConstr(p[i, k] == abs_(z[i, k]))
                        m1.addConstr(M * delta[i, k] + p[i, k] >= 1)
                        m1.addConstr(-M * (1 - delta[i, k]) + p[i, k] <= 1)

            m1.setObjective(obj, GRB.MINIMIZE)
            #print("11111")
            m1.optimize()
            #print("22222")
            end_time = time.time()
            h2 = m1.getVars()
            #print("xxxxxx")
            order = []
            t = []
            contacts = 0
            for item2 in delta.values():
                contacts += item2.x
            for i in range(0, 2*(n+1)):
                if i < n+1:
                    t.append(np.round(h2[i].x, 1))
                else:
                    order.append(h2[i].x)
            output = [order, t, contacts, float(end_time-start_time)]
            return output


# main loop
for length in range(100,200,50):
    l = length/60

    max_ticks = 180  # run simulation for 10 hours, first 2 hours is warmup period

    for itr in range(0, 2):
        num_customers = 0
        bad_customers = 0
        contacts = 0
        times = []
        last = 0
        arrival_times = []
        shopping_times = []
        list_size = []
        speed = []
        time_added = []
        run_time = []
        list_occupied_nodes = []
        Y = np.zeros(N)
        customer_data = []
        for k in range(0, N):
            empty = list([])
            list_occupied_nodes.append(empty)

        for i in range(0, N):
            if len(list_occupied_nodes[i]) == 0:
                Y[i] = 0
                list_occupied_nodes[i].append(0)
            else:
                Y[i] = 1

        for i in range(0, 500):
            r = np.random.uniform(0, 1, 1)
            r = np.random.exponential(1/l,1)[0]
            #inter = -np.log(1-r)/l # why cummulative 
            t = last + r
            t_prime = np.round(t, 1)
            times.append(float(t_prime))
            last = t_prime

        for q in times:
            if q < max_ticks:
                my_list = [0]
                v = 10 * np.random.randint(1, 3, 1)
                if q > 60:
                    arrival_times.append(float(q))
                    num_customers += 1

                # create list of customer
                for i in range(1,N-1): # available nodes
                    p = np.random.uniform(0, 1, 1)
                    if p < 0.2: 
                        my_list.append(i) # selecting the rows with probablity 0.2
                if(len(my_list)>10):
                    my_list = my_list[:10]
                if q > 60:
                    list_size.append(len(my_list))

                c = np.array(C)
                d = list_occupied_nodes.copy()
                y = Y.copy()
                my_list.append(N-1)
                size = len(my_list)
                for i in range(0, N):
                    if not (N - i - 1) in my_list:
                        c = np.delete(c, N - i - 1, 0) # deleting the row of node not to be visited(not in the list)
                        c = np.delete(c, N - i - 1, 1) # deleting the column of node not to be visited(not in the list)
                        d.pop(N - i - 1)
                        y = np.delete(y, N - i - 1, None)
                #print(size)
                #print(c.shape)
                #size = size-1
                result = compute_path(size, c, d, y, q)
                zip1 = zip(result[0], my_list)
                zip2 = zip(result[0], result[1])
                temp1 = [x for _, x in sorted(zip1)]
                temp2 = [x for _, x in sorted(zip2)]
                temp_dict = {"time": q, "travel_order": temp1, "timestamp": temp2}
                print(temp1)
                print(temp2)
                print(result[2])
                customer_data.append(temp_dict)

                if q > 60:
                    end_time = temp2[size - 1] + b + 5 * M[0][temp1[size - 1]] / v
                    arrival_times.append(q)
                    s = np.round(end_time - q, 1)
                    shopping_times.append(float(s))
                    speed.append(float(v))
                    run_time.append(result[3])
                    list_size.append(size)
                    alt = shortest(c, size, v)
                    saved = np.round(s - alt, 1)
                    time_added.append(float(saved))
                    if result[2] > 0:
                        bad_customers += 1
                        contacts += 2*result[2]

                new_blockages = result[1]
                for i in range(1, size):
                    index = my_list[i]
                    list_occupied_nodes[index].append(new_blockages[i])

                for z in range(0, N):
                    for item in list_occupied_nodes[z]:
                        if item + b < q:
                            list_occupied_nodes[z].remove(item)
                for z in range(0, N):
                    if len(list_occupied_nodes[z]) == 0:
                        list_occupied_nodes[z].append(0)

                # print(list_occupied_nodes)
                for z in range(0, N):
                    if list_occupied_nodes[z][0] == 0:
                        Y[z] = 0
                    else:
                        Y[z] = 1

            else:# break when arrival time exceeds max_ticks
                break

        print("itr " + str(itr + 1))
        print(num_customers)

        normalized_shop_t = list(map(truediv, shopping_times, list_size))
        normalized_t_added = list(map(truediv, time_added, list_size))
        norm_speed1 = []
        norm_speed2 = []
        norm_t_added1 = []
        norm_t_added2 = []
        for i in range(0, len(speed)):
            if speed[i] == 10:
                norm_speed1.append(float(normalized_shop_t[i]))
                norm_t_added1.append(float(normalized_t_added[i]))
            else:
                norm_speed2.append(float(normalized_shop_t[i]))
                norm_t_added2.append(float(normalized_t_added[i]))

        row = [l,p,itr + 1, num_customers, mean(norm_speed1), stdev(norm_speed1), mean(norm_speed2),
               stdev(norm_speed2), mean(norm_t_added1), stdev(norm_t_added1), mean(norm_t_added2),
               stdev(norm_t_added2), contacts, mean(run_time), stdev(run_time)]
        Out.loc[len(Out)] = row
        print(row)


# with open('tspmc55.txt', 'w') as outfile:
#     json.dump(Out, outfile)



9
[0, 9, 8, 23, 24, 27, 15, 2, 3, 35]
[0.7, 4.7, 7.7, 10.7, 13.7, 16.7, 19.7, 22.7, 25.7, 28.7]
0.0
10


In [ ]:
Out.to_csv(r'C:\Users\hp\Desktop\rep1.csv',index=True)

10